In [1]:
!pip install langchain-openai \
  langchainhub \
  langchain-chroma \
  langchain-community \
  langchain-text-splitters \
  langchain-cohere \
  tiktoken \
  openai \
  python-dotenv \
  pypdf \
  sentence-transformers \
  ragas \
  xmltodict \
  rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [18]:
# Azure OpenAI credentials
AZURE_OPENAI_ENDPOINT = "https://azureopenai16.openai.azure.com/"
AZURE_OPENAI_API_KEY = "75db73a3b9da40b0b6e0e98273a6029f"
AZURE_OPENAI_API_VERSION = "2024-02-01"

LLM_MODEL = "gpt35turbo"
EMBEDDING_MODEL = "ada0021_6"

COHERE_API_KEY = "BWR8YyveaadqsWa8Ty0FM0vEIAysgJgnjhZVkRP1"

storage_path = "./vectordb"

LANGCHAIN_TRACING_V2 = "true"
LANGCHAIN_API_KEY = "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"

In [2]:
import os
import chromadb
from datasets import Dataset
from dotenv import load_dotenv
from langchain_community.embeddings import SentenceTransformerEmbeddings
from ragas.testset import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas import RunConfig
from ragas.metrics import (context_precision, context_recall)

from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

import os
from langchain_chroma import Chroma
from datasets import load_dataset
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.documents.base import Document

In [3]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

llm = AzureChatOpenAI(
  openai_api_type="azure",
  openai_api_version="2024-02-01",
  openai_api_key=AZURE_OPENAI_API_KEY,
  azure_endpoint=AZURE_OPENAI_ENDPOINT,
  model=LLM_MODEL,
  temperature=0
)

C:\Users\Hammer\Anaconda3\envs\llama\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [4]:
loader = DirectoryLoader("C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()
print('.....document_loaded.....')

.....document_loaded.....


In [5]:
docs

[Document(metadata={'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload\\MACPHERSON, Callum - MSc Dissertation.pdf', 'page': 0}, page_content='Fully automated hand tracking for Parkinson’s Disease\nDiagnosis\nCallum Macpherson\nStudent ID: 201022895\nSupervised by Luisa Cutillo, Samuel Relton, and Hui Fang\nSubmitted in accordance with the requirements for the\nmodule MATH5872M: Dissertation in Data Science and Analytics\nas part of the degree of\nMaster of Science in Data Science and Analytics\nThe University of Leeds, School of Mathematics\nSeptember 2021\nThe candidate conﬁrms that the work submitted is his/her own and that appropriate\ncredit has been given where reference has been made to the work of others.'),
 Document(metadata={'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload\\MACPHERSON, Callum - MSc Dissertation.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\uploa

In [6]:
chunk_size = 1000
chunk_overlap = int(0.15*chunk_size)

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap
)

splits = splitter.split_documents(docs)

In [11]:
from rag.ingestion import create_vector_store, storage_path
from langchain_openai.chat_models import AzureChatOpenAI

vector_store = create_vector_store(splits)

Added 281 chunks to chroma db


In [16]:
from rag.retriever import contextualCompressionRetriever, multiQueryRetriever

retriever = multiQueryRetriever(db_storage_path=storage_path, embedding_function=embedding_function, llm=llm)

In [17]:
retriever.get_relevant_documents("Parkinson")

[Document(metadata={'page': 56, 'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload\\MACPHERSON, Callum - MSc Dissertation.pdf'}, page_content='Each of the 40 Parkinson’s Patients had previously been diagnosed by a consultant neurol-\nogist specialising in movement disorders at Leeds Teaching Hospitals NHS Trust. The Parkin-\nson’s Patients were invited to attend a research clinic appointment speciﬁcally to gather this\ndata set. Each of the diagnosed patients was subjectively and objectively in an ‘on’ motor state,\nwhich means that they met the following criteria:\n1. The patients reported that they felt ‘on’. This is a widely accepted and understood term\nthat refers to patients ability to have an awareness that their medications are working\neffectively and that the symptoms of their PD have been reduced by these medications.\n2. The consultant neurologist reported that the patient looked ‘on’. This term is clinically\naccepted that a trained specialist can obser